# Sample Steam Reviews with word-level RNN with GloVe embeddings

Code inspired from https://github.com/woctezuma/sample-steam-reviews

In [1]:
from google.colab import drive

mount_folder = '/content/gdrive'
drive.mount(mount_folder)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/'

/content/gdrive/My Drive


In [0]:
#!rm -rf sample-steam-reviews/
#!git clone https://github.com/woctezuma/sample-steam-reviews.git

In [4]:
%cd sample-steam-reviews/

/content/gdrive/My Drive/sample-steam-reviews


In [0]:
!git pull
!git checkout stacked-lstm

In [0]:
!pip install -r requirements.txt

In [0]:
!python -m spacy download en_vectors_web_lg

In [0]:
#!python word_level_rnn_with_embeddings.py

In [9]:
from download_review_data import get_artifact_app_id
from export_review_data import get_output_file_name

from word_level_rnn_with_embeddings import train_model, get_vocabulary_file_name, get_examples_of_sentence_start, generic_generate_next

Using TensorFlow backend.


In [0]:
app_id = get_artifact_app_id()
text_file_name = get_output_file_name(app_id)

In [11]:
!ls model.word_level_rnn_with_embeddings.*.hdf5

model.word_level_rnn_with_embeddings.epoch_15.hdf5
model.word_level_rnn_with_embeddings.epoch_16.hdf5
model.word_level_rnn_with_embeddings.epoch_17.hdf5
model.word_level_rnn_with_embeddings.epoch_18.hdf5
model.word_level_rnn_with_embeddings.epoch_19.hdf5
model.word_level_rnn_with_embeddings.epoch_20.hdf5
model.word_level_rnn_with_embeddings.epoch_21.hdf5
model.word_level_rnn_with_embeddings.epoch_22.hdf5


In [0]:
max_sentence_len = 40
overlap_size=35

num_epochs=60
initial_epoch=20
full_model_filename = 'model.word_level_rnn_with_embeddings.epoch_{:02d}.hdf5'.format(initial_epoch)

model, sorted_data_driven_vocabulary = train_model(path=text_file_name,
                                                   max_sentence_len=max_sentence_len,
                                                   overlap_size=overlap_size,
                                                   num_epochs=num_epochs,
                                                   full_model_filename=full_model_filename,
                                                   initial_epoch=initial_epoch)

with open(get_vocabulary_file_name(), 'w', encoding='utf-8') as f:
    print(sorted_data_driven_vocabulary, file=f)


Loading GloVe...

Preparing the sentences...
Num sentences: 136184
Num unique words: 13887
Result embedding shape: (13887, 300)

Preparing the data for LSTM...
train_x shape: (136184, 40)
train_y shape: (136184,)

Training LSTM...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loading model model.word_level_rnn_with_embeddings.epoch_20.hdf5 with initial epoch = 20
Instructions for updating:
Use tf.cast instead.
Epoch 21/60
136184/136184 [==============================] - 250s 2ms/step - loss: 7.1528

Generating text after epoch: 20
i like this game because... -> i like this game because counting cons emulate outcast adjustments aimed cinema reallly 0515 expires
i do not like this game because... -> i do not like this game because mb eveyone tacticle undeniably ahold step reccommend approval weapons afraid
the... -> the redeem refrain nonsensical u

In [0]:
load_previous_vocabulary = False

if load_previous_vocabulary:
    with open(get_vocabulary_file_name(), 'r', encoding='utf-8') as f:
        sorted_data_driven_vocabulary = f.readlines()

num_generated = 10

for text in get_examples_of_sentence_start():
    my_sample = generic_generate_next(text, model, sorted_data_driven_vocabulary, num_generated)
    print('{}... -> {}'.format(text, my_sample))